In [132]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris_data=load_iris()

In [133]:
# Features names and target values
print("Feature names:", iris_data.feature_names)
print("First 5 rows of features:\n", iris_data.data[:5])
print("\nTarget names (species):", iris_data.target_names)
print("First 5 target values:", iris_data.target[:5])

Feature names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
First 5 rows of features:
 [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]

Target names (species): ['setosa' 'versicolor' 'virginica']
First 5 target values: [0 0 0 0 0]


In [134]:
# We split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=0.8, random_state=42)

In [135]:
# We can test a prediction model
# We are going to use SVM:

from sklearn import svm
from sklearn.metrics import accuracy_score

model=svm.SVC(probability=True)
model.fit(X_train,y_train)
y_pred=model.predict_proba(X_test)
print(f"Probability of each class:\n {y_pred}")
y_pred=model.predict(X_test)
accuracy=accuracy_score(y_pred,y_test)
print(f"Accuracy: {accuracy}")

Probability of each class:
 [[0.03419823 0.73449337 0.23130841]
 [0.78621604 0.15564937 0.05813459]
 [0.05456227 0.02725968 0.91817805]
 [0.03594644 0.73482716 0.22922641]
 [0.03764961 0.60087194 0.36147845]
 [0.81972623 0.12906174 0.05121203]
 [0.0884057  0.8522178  0.0593765 ]
 [0.03635684 0.18134151 0.78230165]
 [0.03306672 0.75898322 0.20795006]
 [0.0590621  0.87560148 0.06533643]
 [0.03434195 0.25294749 0.71271056]
 [0.85267215 0.10133242 0.04599542]
 [0.85371085 0.1001512  0.04613795]
 [0.84208595 0.1105837  0.04733035]
 [0.83770453 0.11106158 0.05123389]
 [0.03850916 0.57928437 0.38220647]
 [0.02868143 0.06501812 0.90630045]
 [0.06077015 0.87920337 0.06002648]
 [0.03643266 0.78458682 0.17898053]
 [0.02782464 0.09727133 0.87490403]
 [0.82775611 0.12115212 0.05109177]
 [0.03240902 0.44159566 0.52599532]
 [0.81565266 0.13150767 0.05283967]
 [0.02790876 0.10809082 0.86400042]
 [0.06047293 0.0422708  0.89725626]
 [0.0331092  0.16013345 0.80675735]
 [0.02946535 0.11259249 0.85794216]


The model seems to be extremely efficient to predict the type of petal since the probabilities are really distributed

We are going to save our model in a joblib file

In [136]:
from joblib import dump,load

dump(model,'iris_model.joblib')

['iris_model.joblib']

In [137]:
from flask import Flask, request, jsonify
from joblib import load
import numpy as np

app = Flask(__name__)

# Load the model
model = load('iris_model.joblib')

# Make sure to load the iris dataset for target_names (assuming it's not already loaded)
from sklearn.datasets import load_iris
iris_data = load_iris()

@app.route('/predict', methods=['GET'])
def predict():
    try:
        # Extract features from query parameters
        features = [float(request.args.get(feat)) for feat in ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
        
        # Make prediction - Note the change here to wrap features in another list
        prediction = model.predict([features])[0]
        probas = model.predict_proba([features])[0]  # Wrap features in another list
        species = iris_data.target_names[prediction]

        probabilities_dict = {iris_data.target_names[i]: probas[i] for i in range(len(probas))}

        # Standardized API response format
        response = {
            "prediction": species,
            "probabilities": probabilities_dict
        }

    except Exception as e:
        response = {
            "message": str(e)
        }

    return jsonify(response)

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [05/Feb/2024 16:28:01] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [05/Feb/2024 16:28:05] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2024 16:28:14] "GET /predict?sepal_length=5.1&sepal_width=3.5&petal_length=1.4&petal_width=0.2 HTTP/1.1" 200 -
